In [ ]:
#JANGAN DI RUN DULU SEBELUM SIAP RECORD SEMUA DATA TRAINING
#VERSI MANUAL

import cv2
import mediapipe as mp
import csv
import os
import numpy as np

# 1. KONFIGURASI
FILE_NAME = 'dataset_2_tangan.csv'
FOLDER_NAME = 'data'
CSV_PATH = os.path.join(FOLDER_NAME, FILE_NAME)

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)

# 2. INISIALISASI
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)

# 3. INPUT KATA YANG AKAN DIREKAM
print("--- SISTEM PEREKAMAN SKALA BESAR ---")
label_target = input("Ketik nama kosakata yang mau direkam sekarang: ").strip().upper()
print(f"Siap merekam untuk kata: {label_target}")
print("KONTROL: Tekan 'S' untuk SIMPAN data | Tekan 'Q' untuk SELESAI kata ini")

# Buat Header jika file baru
if not os.path.exists(CSV_PATH):
    header = ['label'] + [f'{c}{i}' for i in range(42) for c in ['x', 'y', 'z']]
    with open(CSV_PATH, mode='w', newline='') as f:
        csv.writer(f).writerow(header)

cap = cv2.VideoCapture(0)
counter = 0 # Menghitung berapa banyak sampel yang sudah diambil

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    key = cv2.waitKey(1) & 0xFF
    
    if results.multi_hand_landmarks:
        koordinat_baris = np.zeros(126).tolist()
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            if idx > 1: break
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            for i, lm in enumerate(hand_landmarks.landmark):
                index_awal = (idx * 63) + (i * 3)
                koordinat_baris[index_awal:index_awal+3] = [lm.x, lm.y, lm.z]

        # REKAM DATA SAAT TEKAN 'S'
        if key == ord('s'):
            with open(CSV_PATH, mode='a', newline='') as f:
                csv.writer(f).writerow([label_target] + koordinat_baris)
            counter += 1
            print(f"Data {label_target} tersimpan! (Total: {counter})")

    # UI INFO
    cv2.putText(frame, f"KATA: {label_target}", (10, 30), 1, 2, (0, 255, 0), 2)
    cv2.putText(frame, f"SAMPEL: {counter}", (10, 70), 1, 2, (255, 255, 0), 2)
    cv2.putText(frame, "Tekan 'S' untuk Rekam | 'Q' untuk Ganti Kata", (10, h-20), 1, 1, (255, 255, 255), 1)
    
    cv2.imshow('Collector Skala Besar', frame)
    if key == ord('q'): break

cap.release()
cv2.destroyAllWindows()

In [19]:
#AUTO VERSION
import cv2
import mediapipe as mp
import csv
import os
import numpy as np
import time

# 1. KONFIGURASI
FILE_NAME = 'dataset_2_tangan.csv'
FOLDER_NAME = 'data'
CSV_PATH = os.path.join(FOLDER_NAME, FILE_NAME)
SAMPLES_NEEDED = 150  # Jumlah sampel otomatis per kata
DELAY_BETWEEN_SAMPLES = 0.1  # Jeda simpan (0.1 detik = sangat cepat)

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)

# 2. INISIALISASI MEDIAPIPE
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)

# 3. INPUT TARGET KATA
print("--- SISTEM AUTO-RECORD DATA ISYARAT ---")
label_target = input("Ketik nama kata yang akan direkam: ").strip().upper()

if not os.path.exists(CSV_PATH):
    header = ['label'] + [f'{c}{i}' for i in range(42) for c in ['x', 'y', 'z']]
    with open(CSV_PATH, mode='w', newline='') as f:
        csv.writer(f).writerow(header)

cap = cv2.VideoCapture(0)
is_recording = False
counter = 0
last_save_time = 0

print(f"\nSiap merekam '{label_target}'")
print("KONTROL: Tekan 'R' untuk MULAI AUTO-RECORD | 'Q' untuk BATAL/KELUAR")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    current_time = time.time()
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('r'): 
        is_recording = True
        print("Perekaman dimulai... Gerakkan tangan Anda!")
    if key == ord('q'): break

    if results.multi_hand_landmarks:
        # Gambar landmark di layar
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # LOGIKA AUTO-RECORD
        if is_recording and counter < SAMPLES_NEEDED:
            # Cek apakah jeda waktu sudah cukup (0.1 detik)
            if current_time - last_save_time >= DELAY_BETWEEN_SAMPLES:
                koordinat_baris = np.zeros(126).tolist()
                for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    if idx > 1: break
                    for i, lm in enumerate(hand_landmarks.landmark):
                        index_awal = (idx * 63) + (i * 3)
                        koordinat_baris[index_awal:index_awal+3] = [lm.x, lm.y, lm.z]

                # Simpan ke CSV
                with open(CSV_PATH, mode='a', newline='') as f:
                    csv.writer(f).writerow([label_target] + koordinat_baris)
                
                counter += 1
                last_save_time = current_time

        # Jika target tercapai
        if counter >= SAMPLES_NEEDED:
            is_recording = False
            cv2.putText(frame, "SELESAI! Tekan Q", (w//4, h//2), 1, 2, (0, 255, 0), 3)

    # --- INFORMASI UI ---
    status = "RECORDING..." if is_recording else "READY (Press R)"
    color = (0, 0, 255) if is_recording else (0, 255, 0)
    
    cv2.rectangle(frame, (0, 0), (w, 50), (30, 30, 30), -1)
    cv2.putText(frame, f"{label_target}: {counter}/{SAMPLES_NEEDED}", (10, 35), 1, 1.5, (255, 255, 255), 2)
    cv2.putText(frame, status, (w-200, 35), 1, 1.5, color, 2)
    
    # Progress Bar sederhana di bawah
    bar_width = int((counter / SAMPLES_NEEDED) * w)
    cv2.rectangle(frame, (0, h-10), (bar_width, h), (0, 255, 255), -1)

    cv2.imshow('Auto-Collector Pro', frame)

cap.release()
cv2.destroyAllWindows()
print(f"\nBerhasil mengumpulkan {counter} sampel untuk kata '{label_target}'")

--- SISTEM AUTO-RECORD DATA ISYARAT ---


Ketik nama kata yang akan direkam:  SIAPA



Siap merekam 'SIAPA'
KONTROL: Tekan 'R' untuk MULAI AUTO-RECORD | 'Q' untuk BATAL/KELUAR
Perekaman dimulai... Gerakkan tangan Anda!

Berhasil mengumpulkan 150 sampel untuk kata 'SIAPA'
